In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import boto3
import logging
from datetime import datetime

boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

<IPython.core.display.Javascript object>

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
# Define the bucket name and region
S3_BUCKET = os.getenv("S3_BUCKET")
S3_PREFIX = os.getenv("S3_PREFIX")
REGION = os.getenv("REGION")
SAGE_MAKER_LOCAL_ROLE = os.getenv("SAGE_MAKER_LOCAL_ROLE")
print(f"S3_BUCKET: {S3_BUCKET}")
print(f"REGION: {REGION}")
print(f"SAGE_MAKER_LOCAL_ROLE: {SAGE_MAKER_LOCAL_ROLE}")

S3_BUCKET: sgmkr-thangtran3112
REGION: us-west-2
SAGE_MAKER_LOCAL_ROLE: arn:aws:iam::654654352356:role/service-role/AmazonSageMaker-ExecutionRole-20250111T085887


<IPython.core.display.Javascript object>

In [5]:
import sagemaker
from sagemaker.transformer import Transformer

<IPython.core.display.Javascript object>

In [6]:
sgmkr_clnt = boto3.client("sagemaker")
sgmkr_rt = boto3.client("runtime.sagemaker")

<IPython.core.display.Javascript object>

In [9]:
# role_arn = sagemaker.get_execution_role()
role_arn = os.getenv("SAGE_MAKER_LOCAL_ROLE")
role_arn

'arn:aws:iam::654654352356:role/service-role/AmazonSageMaker-ExecutionRole-20250111T085887'

<IPython.core.display.Javascript object>

#### Create model

In [11]:
from sagemaker import image_uris

model_img = image_uris.retrieve(
  framework="xgboost", 
  region=REGION,
  version="latest"
)
model_img

[01/11/25 21:00:46] INFO     Ignoring unnecessary instance type: None.                            ]8;id=509496;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=29108;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/image_uris.py#528\528]8;;\

'433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest'

<IPython.core.display.Javascript object>

In [12]:
!aws s3 ls {S3_BUCKET}/{S3_PREFIX}/model/

                           PRE iris-xgboost-2025-01-11-18-03-55/
                           PRE iris-xgboost-2025-01-11-19-59-53/


<IPython.core.display.Javascript object>

In [13]:
model_path = (
    "s3://{}/{}/{}".format(S3_BUCKET, S3_PREFIX, "model/iris-xgboost-2025-01-11-19-59-53/output/model.tar.gz")
)
model_path

's3://sgmkr-thangtran3112/iris/model/iris-xgboost-2025-01-11-19-59-53/output/model.tar.gz'

<IPython.core.display.Javascript object>

In [14]:
model_name = "iris-xgboost-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(model_name)

iris-xgboost-2025-01-11-21-03-57


<IPython.core.display.Javascript object>

In [15]:
response = sgmkr_clnt.create_model(
    ModelName=model_name,
    PrimaryContainer={"Image": model_img, "ModelDataUrl": model_path},
    ExecutionRoleArn=role_arn,
)

print(response)

{'ModelArn': 'arn:aws:sagemaker:us-west-2:654654352356:model/iris-xgboost-2025-01-11-21-03-57', 'ResponseMetadata': {'RequestId': 'a8499189-7b72-4e23-9cd0-e7e09df79717', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a8499189-7b72-4e23-9cd0-e7e09df79717', 'content-type': 'application/x-amz-json-1.1', 'content-length': '94', 'date': 'Sun, 12 Jan 2025 05:04:07 GMT'}, 'RetryAttempts': 0}}


<IPython.core.display.Javascript object>

#### Batch Transform

In [16]:
batch_ip = 's3://{}/{}/{}'.format(S3_BUCKET, S3_PREFIX, 'batch_transform')
batch_op = 's3://{}/{}/{}'.format(S3_BUCKET, S3_PREFIX, 'batch_transform')

<IPython.core.display.Javascript object>

In [28]:
transformer = Transformer(
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=batch_op,
)

transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv"
)
transformer.wait()

.................................
Arguments: serve
[2023-02-03 09:53:36 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2023-02-03 09:53:36 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2023-02-03 09:53:36 +0000] [1] [INFO] Using worker: gevent
[2023-02-03 09:53:36 +0000] [21] [INFO] Booting worker with pid: 21
[2023-02-03 09:53:36 +0000] [22] [INFO] Booting worker with pid: 22
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.py)', 'urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)']. 
  monkey.patch_all(subprocess=True)
[2023-02

UnexpectedStatusException: Error for Transform job xgboost-2023-02-03-09-48-15-211: Failed. Reason: ClientError: See job logs for more information

<IPython.core.display.Javascript object>

#### Endpoint
* Deploy the Inference model from S3 artifact

In [17]:
ep_config_name = "tmp-ep-config-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_config_name)

tmp-ep-config-2025-01-11-21-10-06-527791


<IPython.core.display.Javascript object>

In [18]:
response = sgmkr_clnt.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    ProductionVariants=[
        {
            "VariantName": "version-1",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m4.xlarge",
            # sever_less = ''
        },
    ],
)

print(response)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:654654352356:endpoint-config/tmp-ep-config-2025-01-11-21-10-06-527791', 'ResponseMetadata': {'RequestId': '7db52085-f197-48ef-9b85-83dcc88df064', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7db52085-f197-48ef-9b85-83dcc88df064', 'content-type': 'application/x-amz-json-1.1', 'content-length': '121', 'date': 'Sun, 12 Jan 2025 05:10:09 GMT'}, 'RetryAttempts': 0}}


<IPython.core.display.Javascript object>

In [19]:
ep_name = "tmp-ep-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S-%f")
print(ep_name)

tmp-ep-2025-01-11-21-10-21-248249


<IPython.core.display.Javascript object>

In [20]:
response = sgmkr_clnt.create_endpoint(
    EndpointName=ep_name, EndpointConfigName=ep_config_name,
)
print(response)

{'EndpointArn': 'arn:aws:sagemaker:us-west-2:654654352356:endpoint/tmp-ep-2025-01-11-21-10-21-248249', 'ResponseMetadata': {'RequestId': '1706a55d-f35e-4eec-b641-8c6119f9b1bb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1706a55d-f35e-4eec-b641-8c6119f9b1bb', 'content-type': 'application/x-amz-json-1.1', 'content-length': '101', 'date': 'Sun, 12 Jan 2025 05:10:31 GMT'}, 'RetryAttempts': 0}}


<IPython.core.display.Javascript object>

In [21]:
# Deploy and wait for the endpoint to be in service. It would normally take 5-10 minutes
waiter = sgmkr_clnt.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=ep_name, WaiterConfig={"Delay": 123, "MaxAttempts": 123})
print("Endpoint created")

Endpoint created


<IPython.core.display.Javascript object>

In [22]:
payload = "7.7, 3.0, 6.1, 2.3"
# payload = '7.7, 3.0, 6.1, 2.3 \n 7.9, 3.8, 6.4, 2.1'

<IPython.core.display.Javascript object>

In [23]:
sgmkr_runt = boto3.client("runtime.sagemaker")

<IPython.core.display.Javascript object>

In [24]:
response = sgmkr_runt.invoke_endpoint(
    EndpointName=ep_name, ContentType="text/csv", Body=payload,
)

prediction = response["Body"].read().decode()
print(prediction)

2.0


<IPython.core.display.Javascript object>

In [26]:
print("Deleting sagemaker endpoint")
response = sgmkr_clnt.delete_endpoint(EndpointName = ep_name)  
print("Deleted sagemaker endpoint")
#print(response)    

Deleting sagemaker endpoint
Deleted sagemaker endpoint


<IPython.core.display.Javascript object>

In [27]:
print("Deleting sagemaker endpoint configuration")
response = sgmkr_clnt.delete_endpoint_config(EndpointConfigName = ep_config_name)
print("Deleted sagemaker endpoint configuration")
#print(response)

Deleting sagemaker endpoint configuration
Deleted sagemaker endpoint configuration


<IPython.core.display.Javascript object>